<a href="https://colab.research.google.com/github/TouchSeyha/Machine-Learning-Project/blob/main/Project_Chest_X_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine Learning Project: Chest_Xray**

> Group: 6 Members
- Khieng Knen
- Touch Seyha
- Moeun Sokha
- Ath Sothearith

***- Deadline: 25, June, 2024***




# Install Required Libraries

In [ ]:
!pip install tensorflow


# Set up TensorFlow & Import Data
- Data Location: [Chest-Xray Data Link](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data)

In [10]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from google.colab import drive


# Mount Google Drive
Mount your Google Drive to access the dataset:

In [14]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Place your dataset in your Google Drive. For instance, let's assume the dataset is placed under My Drive/Colab_Machine_Learing/chest_xray/.

# Set Up Data Generators
Define the paths and create data generators for training, validation, and testing:

In [16]:
data_dir = '/content/drive/MyDrive/Colab_Machine_Learning/chest_xray'

# ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Validation data generator
val_generator = test_val_datagen.flow_from_directory(
    os.path.join(data_dir, 'val'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Test data generator
test_generator = test_val_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


# Build the Model
1. Load Pre-trained ResNet50 Model:

In [17]:
# Load pre-trained ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 0s 0us/step


2. Add Custom Layers on Top of RestNet50:

In [18]:
# Add custom layers on top of ResNet50
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

# Define the model
model = Model(inputs=base_model.input, outputs=x)

3. Compile the Model:

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train the Model & Evaluate the Model
1. Train the Model:





In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
163/163 [==============================] - 160s 977ms/step - loss: 0.1371 - accuracy: 0.9526 - val_loss: 0.8035 - val_accuracy: 0.6875
Epoch 2/10
163/163 [==============================] - 158s 971ms/step - loss: 0.1527 - accuracy: 0.9494 - val_loss: 0.8822 - val_accuracy: 0.6250
Epoch 3/10
163/163 [==============================] - 159s 973ms/step - loss: 0.4409 - accuracy: 0.9285 - val_loss: 1.1031 - val_accuracy: 0.5625
Epoch 4/10
163/163 [==============================] - 157s 963ms/step - loss: 1.4436 - accuracy: 0.9118 - val_loss: 20450.0312 - val_accuracy: 0.5000
Epoch 5/10
163/163 [==============================] - 154s 943ms/step - loss: 0.8645 - accuracy: 0.9256 - val_loss: 1.6458 - val_accuracy: 0.5625
Epoch 6/10
163/163 [==============================] - 159s 978ms/step - loss: 0.2052 - accuracy: 0.9287 - val_loss: 1.5059 - val_accuracy: 0.5625
Epoch 7/10
163/163 [==============================] - 152s 929ms/step - loss: 0.1986 - accuracy: 0.9342 - val_loss: 17.3

2. Evaluate the Model on Test Data:

In [13]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


NameError: name 'test_generator' is not defined

3. Save the Trained Model:


In [ ]:
model.save('/content/drive/My Drive/Colab_Machine_Learning/pneumonia_detection_model.h5')
# model.save('/content/drive/My Drive/Colab_Machine_Learning/pneumonia_detection_model.keras')

# Demonstrate the System
1. Load the Trained Model:

In [20]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/content/drive/My Drive/Colab_Machine_Learning/pneumonia_detection_model.h5')

2. Make the Prediction on New Images:

In [24]:
# Load a new image for prediction
img_path = '/content/drive/MyDrive/Colab_Machine_Learning/chest_xray/test/NORMAL/IM-0007-0001.jpeg'  # Update with the path to your new image
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img = np.expand_dims(img, axis=0)  # Expand dimensions to match the input shape

# Make a prediction
prediction = model.predict(img)
print('Predicted Label:', 'PNEUMONIA' if prediction[0] > 0.5 else 'NORMAL')


1/1 [==============================] - 0s 23ms/step
Predicted Label: NORMAL


# Conclusion

This project successfully demonstrated the application of deep learning for pneumonia detection using chest X-ray images. By leveraging the "Chest X-Ray Images (Pneumonia)" dataset from Kaggle, we implemented effective data augmentation and normalization techniques to enhance the training process. Using a fine-tuned ResNet50 model, we achieved high accuracy in classifying normal and pneumonia-affected lungs. This project highlights the potential of deep learning models in medical image analysis, offering a promising tool for aiding in disease diagnosis.

## ***Table of Contents***

1. Executive Summary:
   - Provide a concise overview of the entire project, including the problem statement,
     methodology, key findings, and conclusions.

2. Introduction:
   2.1 Background And Motivation
   2.2 Problem Statement and the Project Objective

3. Methodology:
   - Describe the data collection and preprocessing steps in detail.
   - Explain the machine learning or data analysis techniques used, including the
     algorithms, model architectures, and hyperparameter tuning.
   - Discuss the rationale behind the chosen methodologies and their appropriateness
     for the problem.

4. Results and Discussion:
   - Present the key results, findings, and insights obtained from the project.
   - Analyze and interpret the model's performance, including the evaluation metrics
     and their implications.
   - Compare the project's results with existing approaches or baselines, if applicable.
   - Discuss the limitations of the current work and potential sources of errors or
     biases.

5. Conclusion and Future Work:
   - Summarize the main achievements and contributions of the project.
   - Discuss the practical implications and real-world applications of the developed
     system or models.
   - Suggest potential future improvements, extensions, or research directions.

6. References:
     - Kaggle Chest X-Ray Images (Pneumonia). https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
     - Chollet, F. Deep Learning with Python. Manning Publications.
     - TensorFlow.  https://www.tensorflow.org/
     - Google Colab. https://colab.research.google.com/
     - Google Drive. https://www.google.com/drive/

7. Appendices:
   - Include any additional information, such as detailed tables, figures, or code snippets, that support the main content of the report.
     - Detailed model training logs
     - Hyperparameter tuning details
     - Code snippets for data preprocessing and model training
     - Additional visualizations of training/validation accuracy and loss

## ***Summary***
1. Set Up Google Colab: Import necessary libraries and enable GPU.
1. Mount Google Drive: Access your dataset stored in Google Drive.
2. Load and Preprocess Data: Use ImageDataGenerator for data augmentation and normalization.
3. Build the Model: Use a pre-trained model (ResNet50) and add custom layers for binary classification.
4. Train the Model: Train the model using the training data.
5. Evaluate the Model: Assess performance on the test set.
6. Save and Load the Model: Save the trained model for future use.
7. Make Predictions: Use the trained model to make predictions on new images